In [ ]:
# Importing libraries
import bs4 as bs
import os
import sys
import urllib.request
from PyQt5.QtWebEngineWidgets import QWebEnginePage
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
import pytube # library for downloading youtube videos


class Page(QWebEnginePage):
    def __init__(self, url):
        self.app = QApplication(sys.argv)
        QWebEnginePage.__init__(self)
        self.html = ''
        self.loadFinished.connect(self._on_load_finished)
        self.load(QUrl(url))
        self.app.exec_()

    def _on_load_finished(self):
        self.html = self.toHtml(self.Callable)
        print('Load finished')

    def Callable(self, html_str):
        self.html = html_str
        self.app.quit()

links = []

def exact_link(link):
    vid_id = link.split('=')
    # print(vid_id)
    str = ""
    for i in vid_id[0:2]:
        str += i + "="

    str_new = str[0:len(str) - 1]
    index = str_new.find("&")

    new_link = "https://www.youtube.com" + str_new[0:index]
    return new_link


url = "https://www.youtube.com/playlist?list=LL"
# Scraping and extracting the video
# links from the given playlist url
page = Page(url)
count = 0

soup = bs.BeautifulSoup(page.html, 'html.parser')
playlist_name = soup.find('yt-formatted-string', {'class': 'ytd-playlist-sidebar-primary-info-renderer'}).find('a').text
for link in soup.find_all('a', id='thumbnail'):

    # not using first link because it is
    # playlist link not particular video link
    if count == 0:
        count += 1
        continue
    else:
        try:
            # Prevents error for links with no href.
            vid_src = link['href']
            # print(vid_src)
            # keeping the format of link to be
            # given to pytube otherwise in some cases
            new_link = exact_link(vid_src)

            # error might occur due to this
            # print(new_link)

            # appending the link to the links array
            links.append(new_link)
        except Exception as exp:
            pass # No function necessary for invalid <a> tags.

# make dir
os.chdir('/Users/benai/Desktop/amliyat')
dirs = os.getcwd()
if (os.path.isdir(dirs+'/'+playlist_name) == False):
    os.mkdir(dirs + '/'+playlist_name)
os.chdir(dirs+'/'+playlist_name)

# check if file is already downloaded
exists_links = []
try:
    with open(playlist_name+'.txt') as f:
        lines = f.readlines()
        for i in lines:
            exists_links.append(i.rstrip())
        f.close()
except FileNotFoundError:
    # save playlist with file name
    myfile = open(playlist_name+'.txt', 'w')
    for i in links:
        myfile.write(i + '\n')
    myfile.close()
    
# new list
new_list_to_download = list(set(exists_links) ^ set(links))
    
# add to file if file name does not exists
if (len(new_list_to_download) !=  0):
    myfile = open(playlist_name+'.txt', 'w')
    for i in new_list_to_download:
        myfile.write(i + '\n')
    myfile.close()
    
print("To download {} files {} => ".format(len(new_list_to_download), new_list_to_download))
print("File will be saved in => ", os.getcwd())

# downloading each video from
# the link in the links array
if (len(new_list_to_download) !=0):
    for idx, link in enumerate(new_list_to_download):
        yt = pytube.YouTube(link)

        # Downloaded video will be the best quality video
        stream = yt.streams.filter(progressive=True,
                                file_extension='mp4').order_by(
            'resolution').desc().first()
        try:
            stream.download()
            # printing the links downloaded
            print("Downloaded: ", idx+1, ' => ', link)
        except:
            print('Some error in downloading: ', link)
else:
    print("WARNING: Files already downloaded")

'/Users/benai/Desktop/amliyat/Barket ( money)/Vaishnavi'

dfndjn 12 
